<a id='prerequisits'></a>

# Prerequisits
This section installs `gradslam` (if not already installed), imports the necessary packages for the tutorial, and downloads 'lr kt1' (the first trajectory) of [ICL-NUIM dataset](https://www.doc.ic.ac.uk/~ahanda/VaFRIC/iclnuim.html) and structures it as below:
```
ICL
    living_room_traj1_frei_png
        depth/    rgb/    associations.txt    livingRoom1.gt.freiburg    livingRoom1n.gt.sim
```


We set the ICL path variable: `icl_path='ICL/'`. The ICL data is loaded into the following variables: <br>

* `colors`: of shape (batch_size, sequence_length, height, width, 3) <br>
* `depths`: of shape (batch_size, sequence_length, height, width, 1) <br>
* `intrinsics`: of shape (batch_size, 1, 4, 4) <br>
* `poses`: of shape (batch_size, sequence_length, 4, 4) <br>

Finally `RGBDImages` is created from the ICL data and visualized.

In [ ]:
# install gradslam (if not installed)
try:
    import gradslam as gs
except ImportError:
    print("Installing gradslam...")
    !pip install 'git+https://github.com/gradslam/gradslam.git' -q
    print('Installed')

# import necessary packages
import gradslam as gs
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from gradslam import Pointclouds, RGBDImages
from gradslam.datasets import ICL, TUM
from gradslam.slam import PointFusion
from torch.utils.data import DataLoader
from pytorch3d.loss.chamfer import chamfer_distance
from time import time

In [ ]:
# download 'lr kt1' of ICL dataset
if not os.path.isdir('ICL'):
    os.mkdir('ICL')
if not os.path.isdir('ICL/living_room_traj1_frei_png'):
    print('Downloading ICL/living_room_traj1_frei_png dataset...')
    os.mkdir('ICL/living_room_traj1_frei_png')
    !wget http://www.doc.ic.ac.uk/~ahanda/living_room_traj1_frei_png.tar.gz -P ICL/living_room_traj1_frei_png/ -q
    !tar -xzf ICL/living_room_traj1_frei_png/living_room_traj1_frei_png.tar.gz -C ICL/living_room_traj1_frei_png/
    !rm ICL/living_room_traj1_frei_png/living_room_traj1_frei_png.tar.gz
    !wget https://www.doc.ic.ac.uk/~ahanda/VaFRIC/livingRoom1n.gt.sim -P ICL/living_room_traj1_frei_png/ -q
    print('Downloaded.')
icl_path = 'ICL/'

# load dataset
dataset = ICL(icl_path, seqlen=8, height=240, width=320)
loader = DataLoader(dataset=dataset, batch_size=2)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses)
rgbdimages.plotly(0).update_layout(autosize=False, height=600, width=400).show()

# Basic PointFusion

> **_NOTE:_**  Make sure to have ran the [prerequisits](#Prerequisits) section before running this section.

This section demonstrates the basic use of PointFusion.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
slam = PointFusion(device=device)
pointclouds, recovered_poses = slam(rgbdimages)
pointclouds.plotly(0, max_num_points=15000).update_layout(autosize=False, width=600).show()

# Step by step PointFusion

> **_NOTE:_**  Make sure to have ran the [prerequisits](#Prerequisits) section before running this section.

This section demonstrates building the pointcloud map from one frame at a time by calling the SLAM object's `.step()` method.

In [ ]:
# load dataset
dataset = ICL(icl_path, seqlen=8, height=240, width=320)
loader = DataLoader(dataset=dataset, batch_size=1)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics)

# step by step SLAM
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
slam = PointFusion(device=device)

pointclouds = Pointclouds(device=device)
batch_size, seq_len = rgbdimages.shape[:2]
initial_poses = torch.eye(4, device=device).view(1, 1, 4, 4).repeat(batch_size, 1, 1, 1)
prev_frame = None
for s in range(seq_len):
    t0 = time()
    live_frame = rgbdimages[:, s].to(device)
    if s == 0 and live_frame.poses is None:
        live_frame.poses = initial_poses
    pointclouds, live_frame.poses = slam.step(pointclouds, live_frame, prev_frame)
#     print(live_frame.poses.shape, pointclouds.points_padded.shape)
    print(f'SLAM step took {(time()-t0):.3f} sec')
    print(f'Image shape: {live_frame.rgb_image.shape}')
    prev_frame = live_frame
pointclouds.plotly(0, max_num_points=15000).update_layout(autosize=False, width=600).show()

# Advanced visualization

> **_NOTE:_**  Make sure to have ran the [prerequisits](#Prerequisits) section before running this section.

This section demonstrates visualization of the pointcloud map as it gets updated from new rgbd frames. It also visualizes the poses with frustums in the 3d map. We use ground truth poses here (`odom=gt`) as the data sequences were fetched with a large dilation value (i.e. small fps) and ICP/gradICP won't work well in this case.

In [ ]:
import numpy as np
import plotly.graph_objects as go

def plotly_map_update_visualization(intermediate_pcs, poses, K, max_points_per_pc=50000, ms_per_frame=50):
    """
    Args:
        - intermediate_pcs (List[gradslam.Pointclouds]): list of gradslam.Pointclouds objects, each of batch size 1
        - poses (torch.Tensor): poses for drawing frustums
        - K (torch.Tensor): Intrinsics matrix
        - max_points_per_pc (int): maximum number of points to plot for each pointcloud
        - ms_per_frame (int): miliseconds per frame for the animation

    Shape:
        - poses: :math:`(L, 4, 4)`
        - K: :math:`(4, 4)`
    """
    def plotly_poses(poses, K):
        """
        Args:
            poses (np.ndarray):
            K (np.ndarray):

        Shapes:
            - poses: :math:`(L, 4, 4)`
            - K: :math:`(4, 4)`
        """
        fx = abs(K[0, 0])
        fy = abs(K[1, 1])
        f = (fx + fy) / 2
        cx = K[0, 2]
        cy = K[1, 2]

        cx = cx / f
        cy = cy / f
        f = 1.

        pos_0 = np.array([0., 0., 0.])
        fustum_0 = np.array(
            [
                [-cx, -cy, f],
                [cx, -cy, f],
                list(pos_0),
                [-cx, -cy, f],
                [-cx, cy, f],
                list(pos_0),
                [cx, cy, f],
                [-cx, cy, f],
                [cx, cy, f],
                [cx, -cy, f],
            ]
        )

        traj = []
        traj_frustums = []
        for pose in poses:
            rot = pose[:3, :3]
            tvec = pose[:3, 3]

            fustum_i = fustum_0 @ rot.T
            fustum_i = fustum_i + tvec
            pos_i = pos_0 + tvec

            pos_i = np.round(pos_i, decimals=2)
            fustum_i = np.round(fustum_i, decimals=2)

            traj.append(pos_i)
            traj_array = np.array(traj)
            traj_frustum = [
                go.Scatter3d(
                    x=fustum_i[:, 0], y=fustum_i[:, 1], z=fustum_i[:, 2],
                    marker=dict(
                        size=0.1,
                    ),
                    line=dict(
                        color='purple',
                        width=4,
                    )
                ),
                go.Scatter3d(
                    x=pos_i[None, 0], y=pos_i[None, 1], z=pos_i[None, 2],
                    marker=dict(
                        size=6.,
                        color='purple',
                    )
                ),
                go.Scatter3d(
                    x=traj_array[:, 0], y=traj_array[:, 1], z=traj_array[:, 2],
                    marker=dict(
                        size=0.1,
                    ),
                    line=dict(
                        color='purple',
                        width=2,
                    )
                ),
            ]
            traj_frustums.append(traj_frustum)
        return traj_frustums

    def frame_args(duration):
        return {
            "frame": {"duration": duration, "redraw": True},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

    # visualization
    scatter3d_list = [pc.plotly(0, as_figure=False, max_num_points=max_points_per_pc) for pc in intermediate_pcs]
    traj_frustums = plotly_poses(poses.cpu().numpy(), K.cpu().numpy())
    data = [[*frustum, scatter3d] for frustum, scatter3d in zip(traj_frustums, scatter3d_list)]

    steps = [
        {"args": [[i], frame_args(0)], "label": i, "method": "animate"}
        for i in range(seq_len)
    ]
    sliders = [
        {
            "active": 0,
            "yanchor": "top",
            "xanchor": "left",
            "currentvalue": {"prefix": "Frame: "},
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": steps,
        }
    ]
    updatemenus = [
        {
            "buttons": [
                {
                    "args": [None, frame_args(ms_per_frame)],
                    "label": "&#9654;",
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],
                    "label": "&#9724;",
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top",
        }
    ]

    fig = go.Figure()
    frames = [{"data": frame, "name": i} for i, frame in enumerate(data)]
    fig.add_traces(frames[0]["data"])
    fig.update(frames=frames)
    fig.update_layout(
        updatemenus=updatemenus,
        sliders=sliders,
        showlegend=False,
        scene=dict(
            xaxis=dict(showticklabels=False, showgrid=False, zeroline=False, visible=False,),
            yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, visible=False,),
            zaxis=dict(showticklabels=False, showgrid=False, zeroline=False, visible=False,),
        )
    )
    fig.show()
    return fig

### GT odom

In [ ]:
dataset = ICL(icl_path, seqlen=300, dilation=0, height=120, width=160)
loader = DataLoader(dataset=dataset, batch_size=1)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses)

# step by step SLAM and store intermediate maps
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
slam = PointFusion(odom='gt', device=device)  # use gt poses because large dilation (small fps) makes ICP difficult
pointclouds = Pointclouds(device=device)
batch_size, seq_len = rgbdimages.shape[:2]
initial_poses = torch.eye(4, device=device).view(1, 1, 4, 4).repeat(batch_size, 1, 1, 1)
prev_frame = None
intermediate_pcs = []
intermediate_poses = []
for s in range(seq_len):
    live_frame = rgbdimages[:, s].to(device)
    if s == 0 and live_frame.poses is None:
        live_frame.poses = initial_poses
    pointclouds, live_frame.poses = slam.step(pointclouds, live_frame, prev_frame)
    prev_frame = live_frame if slam.odom != 'gt' else None
    intermediate_pcs.append(pointclouds[0])
    intermediate_poses.append(live_frame.poses[0])

In [ ]:
# visualize
# rgbdimages.plotly(0).update_layout(autosize=False, height=600, width=400).show()
# fig = plotly_map_update_visualization(intermediate_pcs, poses[0], intrinsics[0, 0], 10000)

### GradICP

In [ ]:
# step by step SLAM and store intermediate maps
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
slam = PointFusion(odom='gradicp', device=device)
pointclouds = Pointclouds(device=device)
batch_size, seq_len = rgbdimages.shape[:2]
initial_poses = torch.eye(4, device=device).view(1, 1, 4, 4).repeat(batch_size, 1, 1, 1)
prev_frame = None
intermediate_pcs_pred = []
intermediate_poses_pred = []
for s in range(seq_len):
    live_frame = rgbdimages[:, s].to(device)
    if s == 0 and live_frame.poses is None:
        live_frame.poses = initial_poses
    pointclouds, live_frame.poses = slam.step(pointclouds, live_frame, prev_frame)
    prev_frame = live_frame
    intermediate_pcs_pred.append(pointclouds[0])
    intermediate_poses_pred.append(live_frame.poses[0])

In [ ]:
# visualize
# rgbdimages.plotly(0).update_layout(autosize=False, height=600, width=400).show()
# fig = plotly_map_update_visualization(intermediate_pcs_pred,
#                                       torch.cat(intermediate_poses_pred, dim=0),
#                                       intrinsics[0, 0], 10000)

### Compare Maps (constructed using traj_slam or traj_gt)

In [ ]:
cd, _ = chamfer_distance(intermediate_pcs[-1].points_padded, intermediate_pcs_pred[-1].points_padded)
print(f'Chamfer distance: {cd.item()}')

In [ ]:
intermediate_pcs[-1].points_padded.shape, intermediate_pcs_pred[-1].points_padded.shape

### Compare trajectories (traj_slam and traj_gt)

In [ ]:
traj_gt = poses[0, :, :3, 3].cpu().numpy()
traj = torch.cat(intermediate_poses_pred, dim=0)[:, :3, 3].cpu().numpy()
traj.shape, traj_gt.shape

In [ ]:
print(f'MSE: {np.linalg.norm(traj - traj_gt) / len(traj)}')

In [ ]:
def plot_trajs(traj, traj_gt):
    
    def set_axis_equal(ax, x_limits=None, y_limits=None, z_limits=None):
        '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
        cubes as cubes, etc..  This is one possible solution to Matplotlib's
        ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

        Input
          ax: a matplotlib axis, e.g., as output from plt.gca().
        '''

        if x_limits is None:
            x_limits = ax.get_xlim3d()
        if y_limits is None:
            y_limits = ax.get_ylim3d()
        if z_limits is None:
            z_limits = ax.get_zlim3d()

        x_range = abs(x_limits[1] - x_limits[0])
        x_middle = np.mean(x_limits)
        y_range = abs(y_limits[1] - y_limits[0])
        y_middle = np.mean(y_limits)
        z_range = abs(z_limits[1] - z_limits[0])
        z_middle = np.mean(z_limits)

        # The plot bounding box is a sphere in the sense of the infinity
        # norm, hence I call half the max range the plot radius.
        plot_radius = 0.5*max([x_range, y_range, z_range])

        ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
        ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
        ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    set_axis_equal(ax, x_limits=[np.min(traj[:,0]), np.max(traj[:,0])],
                       y_limits=[np.min(traj[:,1]), np.max(traj[:,1])],
                       z_limits=[np.min(traj[:,2]), np.max(traj[:,2])])
    ax.set_xlabel('X, [m]')
    ax.set_ylabel('Y, [m]')
    ax.set_zlabel('Z, [m]')
    ax.plot(traj[:,0], traj[:,1], traj[:,2], linewidth=3, label='traj_slam')
    ax.plot(traj_gt[:,0], traj_gt[:,1], traj_gt[:,2], linewidth=3, label='traj_gt')
    ax.legend()
    plt.show()

In [ ]:
plot_trajs(traj, traj_gt)

### TUM dataset

In [ ]:
# download 'xyz' of TUM dataset
if not os.path.isdir('TUM'):
    os.mkdir('TUM')
if not os.path.isdir('TUM/rgbd_dataset_freiburg1_xyz'):
    print('Downloading TUM/rgbd_dataset_freiburg1_xyz dataset...')
    !wget https://vision.in.tum.de/rgbd/dataset/freiburg1/rgbd_dataset_freiburg1_xyz.tgz -P TUM/ -q
    !tar -xzf TUM/rgbd_dataset_freiburg1_xyz.tgz -C TUM/
    !rm TUM/rgbd_dataset_freiburg1_xyz.tgz
    print('Downloaded.')
tum_path = 'TUM/'

In [ ]:
# load dataset
dataset = TUM(tum_path, seqlen=20)
loader = DataLoader(dataset=dataset, batch_size=1)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
slam = PointFusion(odom='gradicp', dsratio=4, device=device)
pointclouds, recovered_poses = slam(rgbdimages)

In [ ]:
slam = PointFusion(odom='gt', dsratio=4, device=device)
pointclouds_gt, recovered_poses_gt = slam(rgbdimages)

In [ ]:
pointclouds.plotly(0).show()

Metrics

In [ ]:
traj = recovered_poses[0, :, :3, 3]
traj_gt = recovered_poses_gt[0, :, :3, 3]
print(f'MSE: {torch.linalg.norm(traj - traj_gt) / len(traj)}')

In [ ]:
plot_trajs(traj.cpu().numpy(), traj_gt.cpu().numpy())

In [ ]:
cd, _ = chamfer_distance(pointclouds.points_padded, pointclouds_gt.points_padded)
print(f'Chamfer distance: {cd.item()}')

In [ ]:
pointclouds.points_padded.shape

### Read data from ROS bag file

In [ ]:
# icl_path = "../../ros_ws/src/gradslam_ros/data/explorer_x1_rgbd_traj/"
icl_path = "../../ros_ws/src/gradslam_ros/data/test/"

dataset = ICL(icl_path, seqlen=80, height=240, width=320)

loader = DataLoader(dataset=dataset, batch_size=1)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses)
rgbdimages.plotly(0).update_layout(autosize=False, height=600, width=400).show()

In [ ]:
colors.shape, depths.shape, poses.shape

In [ ]:
device = torch.device('cuda:0')
slam = PointFusion(device=device)
pointclouds, recovered_poses = slam(rgbdimages)
pointclouds.plotly(0, max_num_points=15000).update_layout(autosize=False, width=600).show()

In [ ]:
recovered_poses.shape

In [ ]:
traj = recovered_poses[0, :, :3, 3].cpu().numpy()

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
# set_axis_equal(ax, x_limits=[np.min(traj[:,0]), np.max(traj[:,0])],
#                    y_limits=[np.min(traj[:,1]), np.max(traj[:,1])],
#                    z_limits=[np.min(traj[:,2]), np.max(traj[:,2])])
ax.set_xlabel('X, [m]')
ax.set_ylabel('Y, [m]')
ax.set_zlabel('Z, [m]')
ax.plot(traj[:,0], traj[:,1], traj[:,2], linewidth=3, label='traj_slam')
ax.legend()
plt.show()